In [1]:
import numpy as np
import scipy #only sparse matrices allowed
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost.sklearn import XGBClassifier

In [4]:
#We need to increase the categorization accuracy.

In [5]:
df_train = pd.read_csv(r"./ift3395-6390-weatherevents/train.csv")

In [6]:
submit_df = pd.read_csv(r"./ift3395-6390-weatherevents/test.csv")

In [7]:
df_train.head()

,S.No,lat,lon,TMQ,U850,V850,UBOT,VBOT,QREFHT,PS,...,T200,T500,PRECT,TS,TREFHT,Z1000,Z200,ZBOT,time,LABELS
0,0,-24.758801,242.1875,16.019615,-4.391696,4.777769,-6.388222,7.725320,0.010277,102326.2344,...,221.594223,261.823212,6.660000e-09,295.622253,293.301514,197.698624,12234.78809,64.372635,20030804,0
1,1,23.820078,277.8125,47.802036,8.623652,9.308566,4.596105,9.938286,0.018531,100803.1094,...,220.343719,269.106720,3.670000e-16,302.496826,302.654083,71.571518,12463.97266,66.834389,20071006,1
2,2,23.820078,276.8750,11.556691,-2.483993,-6.009627,-3.503036,-5.921963,0.007468,101885.3906,...,215.933487,259.610718,0.000000e+00,299.854919,293.339996,160.414246,12099.36133,64.209663,20020331,0
3,3,13.494133,253.1250,53.186630,0.150933,-1.319407,3.757741,-2.172120,0.018809,100824.6328,...,218.457672,267.374329,9.800000e-08,301.606934,299.973450,72.840614,12379.26855,66.243294,20000906,0
4,4,-24.524120,241.2500,23.353998,-7.467506,-5.113565,-9.545109,-4.691221,0.011672,102553.8984,...,214.316605,263.575409,6.410000e-09,295.819488,295.101257,218.449570,12261.22656,64.876427,20000906,0


Let's drop the S.No and separate the labels first of all

In [8]:
#df_train2 = df_train.drop(columns=["S.No"])
#submit_df = submit_df.drop(columns=["S.No"])
#just to make doubly or triply sure that we don't lose these.
labels = df_train[["LABELS"]]

In [9]:
df_train.head()

,S.No,lat,lon,TMQ,U850,V850,UBOT,VBOT,QREFHT,PS,...,T200,T500,PRECT,TS,TREFHT,Z1000,Z200,ZBOT,time,LABELS
0,0,-24.758801,242.1875,16.019615,-4.391696,4.777769,-6.388222,7.725320,0.010277,102326.2344,...,221.594223,261.823212,6.660000e-09,295.622253,293.301514,197.698624,12234.78809,64.372635,20030804,0
1,1,23.820078,277.8125,47.802036,8.623652,9.308566,4.596105,9.938286,0.018531,100803.1094,...,220.343719,269.106720,3.670000e-16,302.496826,302.654083,71.571518,12463.97266,66.834389,20071006,1
2,2,23.820078,276.8750,11.556691,-2.483993,-6.009627,-3.503036,-5.921963,0.007468,101885.3906,...,215.933487,259.610718,0.000000e+00,299.854919,293.339996,160.414246,12099.36133,64.209663,20020331,0
3,3,13.494133,253.1250,53.186630,0.150933,-1.319407,3.757741,-2.172120,0.018809,100824.6328,...,218.457672,267.374329,9.800000e-08,301.606934,299.973450,72.840614,12379.26855,66.243294,20000906,0
4,4,-24.524120,241.2500,23.353998,-7.467506,-5.113565,-9.545109,-4.691221,0.011672,102553.8984,...,214.316605,263.575409,6.410000e-09,295.819488,295.101257,218.449570,12261.22656,64.876427,20000906,0


What even is logistic regression?<br>
It's a linear classification algorithm, which uses a sigmoid activation function.<br>

In [10]:
#https://datascience.stackexchange.com/questions/13567/ways-to-deal-with-longitude-latitude-feature
#https://heartbeat.comet.ml/working-with-geospatial-data-in-machine-learning-ad4097c7228d
#
df_train

,S.No,lat,lon,TMQ,U850,V850,UBOT,VBOT,QREFHT,PS,...,T200,T500,PRECT,TS,TREFHT,Z1000,Z200,ZBOT,time,LABELS
0,0,-24.758801,242.1875,16.019615,-4.391696,4.777769,-6.388222,7.725320,0.010277,102326.2344,...,221.594223,261.823212,6.660000e-09,295.622253,293.301514,197.698624,12234.78809,64.372635,20030804,0
1,1,23.820078,277.8125,47.802036,8.623652,9.308566,4.596105,9.938286,0.018531,100803.1094,...,220.343719,269.106720,3.670000e-16,302.496826,302.654083,71.571518,12463.97266,66.834389,20071006,1
2,2,23.820078,276.8750,11.556691,-2.483993,-6.009627,-3.503036,-5.921963,0.007468,101885.3906,...,215.933487,259.610718,0.000000e+00,299.854919,293.339996,160.414246,12099.36133,64.209663,20020331,0
3,3,13.494133,253.1250,53.186630,0.150933,-1.319407,3.757741,-2.172120,0.018809,100824.6328,...,218.457672,267.374329,9.800000e-08,301.606934,299.973450,72.840614,12379.26855,66.243294,20000906,0
4,4,-24.524120,241.2500,23.353998,-7.467506,-5.113565,-9.545109,-4.691221,0.011672,102553.8984,...,214.316605,263.575409,6.410000e-09,295.819488,295.101257,218.449570,12261.22656,64.876427,20000906,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47755,47755,-30.625815,353.4375,14.680613,3.153518,4.250256,-0.406992,3.480553,0.008333,102109.4844,...,213.804184,259.846222,2.470000e-09,291.791870,290.141663,177.677322,12058.16016,63.678669,20001106,2
47756,47756,-24.993481,242.1875,29.540585,9.181329,-0.369805,7.125068,-5.466469,0.013645,101734.2813,...,213.867416,263.996307,2.620000e-09,296.322327,296.220642,150.108871,12272.25781,65.238327,20010712,2
47757,47757,-30.625815,353.4375,20.991110,7.709649,-6.284292,3.702500,-7.359541,0.010529,101800.8516,...,216.394852,264.192566,4.240000e-09,290.757996,290.736847,152.666443,12232.26953,63.929737,20011029,0
47758,47758,23.585398,276.5625,45.689816,-3.018448,0.280351,-6.938124,0.681055,0.020653,101641.2266,...,217.026199,265.778900,8.620000e-08,302.751862,301.542786,145.255554,12404.29297,66.656548,20020712,0


In [11]:
#https://datascience.stackexchange.com/questions/13567/ways-to-deal-with-longitude-latitude-feature
#https://heartbeat.comet.ml/working-with-geospatial-data-in-machine-learning-ad4097c7228d
#https://stackoverflow.com/questions/14059094/i-want-to-multiply-two-columns-in-a-pandas-dataframe-and-add-the-result-into-a-n
df_train['cos_lat'] = np.cos(df_train[['lat']])
df_train['cos_long'] = np.cos(df_train[['lon']])
df_train['sin_lat'] = np.sin(df_train[['lat']])
df_train['sin_long'] = np.sin(df_train[['lon']])

df_train['x'] = df_train['cos_lat'] * df_train['cos_long']
df_train['y'] = df_train['cos_lat'] * df_train['sin_long']
df_train['z'] = df_train['sin_lat']

#df_train = df_train.drop(columns=['cos_lat','cos_long','sin_lat','sin_long', 'lat', 'lon'])
df_train = df_train.drop(columns=['cos_lat','cos_long','sin_lat','sin_long'])


# df_train["x"] = np.cos(df_train[['lat']]) * np.cos(df_train[['lon']])
# df_train["y"] = np.cos(df_train[['lat']]) * np.sin(df_train[['lon']])
# df_train["z"] = np.sin(df_train[['lat']])

In [12]:
# df_train["rad_lat"] = np.radians(df_train[['lat']])
# df_train["rad_lon"] = np.radians(df_train[['lon']])

In [18]:
df_train.nunique()

S.No      47760
lat          24
lon          35
TMQ       22540
U850      22555
V850      22558
UBOT      22556
VBOT      22555
QREFHT    22533
PS        21520
PSL       21537
T200      22068
T500      21789
PRECT      6506
TS         9827
TREFHT    21892
Z1000     22535
Z200      21880
ZBOT      21964
time        188
LABELS        3
x           120
y           120
z            24
dtype: int64

In [19]:
#Trying clustering:
from sklearn.cluster import KMeans ,AgglomerativeClustering

# creates 5 clusters using hierarchical clustering.
agc = AgglomerativeClustering(n_clusters = 5, affinity='euclidean', linkage='ward')
df_train['cluster_lat'] = agc.fit_predict(df_train[['lat']])


In [ ]:
agc = AgglomerativeClustering(n_clusters = 6, affinity='euclidean', linkage='ward')
df_train['cluster_lat'] = agc.fit_predict(df_train[['lon']])

In [ ]:
plt.figure(figsize = (15,8))
sns.scatterplot(df_train['lat'], df_train['lon'])
plt.show()

In [20]:
df_train

,S.No,lat,lon,TMQ,U850,V850,UBOT,VBOT,QREFHT,PS,...,TREFHT,Z1000,Z200,ZBOT,time,LABELS,x,y,z,cluster_lat
0,0,-24.758801,242.1875,16.019615,-4.391696,4.777769,-6.388222,7.725320,0.010277,102326.2344,...,293.301514,197.698624,12234.78809,64.372635,20030804,0,-0.893379,-0.261608,0.365287,3
1,1,23.820078,277.8125,47.802036,8.623652,9.308566,4.596105,9.938286,0.018531,100803.1094,...,302.654083,71.571518,12463.97266,66.834389,20071006,1,0.055323,0.249209,-0.966868,0
2,2,23.820078,276.8750,11.556691,-2.483993,-6.009627,-3.503036,-5.921963,0.007468,101885.3906,...,293.339996,160.414246,12099.36133,64.209663,20020331,0,0.233623,0.102889,-0.966868,0
3,3,13.494133,253.1250,53.186630,0.150933,-1.319407,3.757741,-2.172120,0.018809,100824.6328,...,299.973450,72.840614,12379.26855,66.243294,20000906,0,-0.134827,0.584272,0.800280,2
4,4,-24.524120,241.2500,23.353998,-7.467506,-5.113565,-9.545109,-4.691221,0.011672,102553.8984,...,295.101257,218.449570,12261.22656,64.876427,20000906,0,-0.651826,0.498236,0.571737,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47755,47755,-30.625815,353.4375,14.680613,3.153518,4.250256,-0.406992,3.480553,0.008333,102109.4844,...,290.141663,177.677322,12058.16016,63.678669,20001106,2,-0.005860,0.703742,0.710432,1
47756,47756,-24.993481,242.1875,29.540585,9.181329,-0.369805,7.125068,-5.466469,0.013645,101734.2813,...,296.220642,150.108871,12272.25781,65.238327,20010712,2,-0.950409,-0.278308,0.138810,3
47757,47757,-30.625815,353.4375,20.991110,7.709649,-6.284292,3.702500,-7.359541,0.010529,101800.8516,...,290.736847,152.666443,12232.26953,63.929737,20011029,0,-0.005860,0.703742,0.710432,1
47758,47758,23.585398,276.5625,45.689816,-3.018448,0.280351,-6.938124,0.681055,0.020653,101641.2266,...,301.542786,145.255554,12404.29297,66.656548,20020712,0,0.023328,0.002396,-0.999725,0


# Normalized and Standardized datasets:

In [85]:
def df_standardizer(df_train):
    #df_train is the train df loaded from our csv file
    #labels are not dealt with here, so keep them in a separate column before calling this
    df_train2 = df_train.drop(columns=["S.No"])
    
    #https://stackoverflow.com/questions/22005911/convert-columns-to-string-in-pandas
    #converting to string for the split
    df_train2["time"] = df_train2["time"].astype(str)
    #expand will return multiple columns with the split strings
    split_columns = df_train2["time"].str.split("",expand=True)
    year = split_columns[1] + split_columns[2] + split_columns[3] +split_columns[4]
    month = split_columns[5] + split_columns[6]
    date = split_columns[7] + split_columns[8]
    
    #converting series to df and renaming the column so we dont end up with a bunch of columns named 0
    #https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html
    year = pd.DataFrame(year)
    month = pd.DataFrame(month)
    date = pd.DataFrame(date)

    year = year.rename(columns={0: "year"})
    month = month.rename(columns={0: "month"})
    date = date.rename(columns={0: "date"})

    #we need to convert the strings back to integers
    #https://datatofish.com/string-to-integer-dataframe/
    year['year'] = year['year'].astype(int)
    month['month'] = month['month'].astype(int)
    date['date'] = date['date'].astype(int)
    
    #adding these back to the original df
    df = pd.concat([df_train2, year, month, date], axis = 1)
    df = df.drop(columns=["time"])
    #df.head()
    
    standardized_df = (df-df.mean())/df.std()
    #standardized_df.head()
    #standardized_df = standardized_df.drop(columns=["LABELS"])
    return standardized_df

In [86]:
df_train = df_standardizer(df_train)
df_train = df_train.drop(columns=["LABELS"])

In [87]:
def df_to_train_test_split(df,labels):
    #df is the normalized or standardized df
    #labels are just the labels.
#         So we need 6760 test exmaples, in 18:4:1 ratio roughly to match the train test.<br>
#         So we need:<br>
#         1s - 294<br>
#         2s - 1176<br>
#         0s - 5290<br>
    data_with_labels = pd.concat([df, labels],axis = 1)
    data_with_labels_0s = data_with_labels[data_with_labels["LABELS"] == 0]
    data_with_labels_1s = data_with_labels[data_with_labels["LABELS"] == 1]
    data_with_labels_2s = data_with_labels[data_with_labels["LABELS"] == 2]

    data_1_test = data_with_labels_1s.iloc[0:294]
    data_2_test = data_with_labels_2s.iloc[0:1176]
    data_0_test = data_with_labels_0s.iloc[0:5290]

    data_1_train = data_with_labels_1s.iloc[294:]
    data_2_train = data_with_labels_2s.iloc[1176:]
    data_0_train = data_with_labels_0s.iloc[5290:]

    test = pd.concat([data_1_test, data_2_test, data_0_test])
    #Houston, we have a problem, we need to shuffle the indices, but pandas has us covered
    #https://stackoverflow.com/questions/29576430/shuffle-dataframe-rows
    train = pd.concat([data_1_train, data_2_train, data_0_train])

    #Shuffling:
    #df = df.sample(frac=1).reset_index(drop=True)
    test = test.sample(frac=1).reset_index(drop=True)
    train = train.sample(frac=1).reset_index(drop=True)

    #Finally, we have our train and test sets.
    #Let's try to follow this post: https://github.com/rahulrrai/multinomial-logistic-regression/blob/master/multinomial-logistic-regression.ipynb
    train_np = train.to_numpy()
    test_np = test.to_numpy()

    #number of training examples
    #this is 41000 in our case
    num_of_train = train_np.shape[0]
    num_of_test = test_np.shape[0]
    #retaining just the features
    X_train = train_np[:,:-1]
    X_test = test_np[:,:-1]
    ##As we are using vector implementation adding a column in X with all the value 1
    # I'm guessing this is the bias term added to our train matrix itself
    X_train = np.hstack((np.ones((num_of_train,1)),X_train))
    X_test = np.hstack((np.ones((num_of_test,1)), X_test))
    #output 
    y_train = train_np[:,-1:]
    y_test = test_np[:,-1:]
    #each individual entry in y is a numpy array, let's see if this works out.
    
    return X_train, X_test, y_train, y_test

In [88]:
X_train, X_test, y_train, y_test = df_to_train_test_split(df_train, labels)

In [89]:
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [90]:
y_train.ravel().shape

(41000,)

In [91]:
#https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
logreg_clf = LogisticRegression(solver='saga', max_iter = 10000, class_weight = None, C= 1.5).fit(X_train,y_train.ravel())
logreg_predictions = logreg_clf.predict(X_test)
logreg_accuracy = logreg_clf.score(X_test, y_test.ravel())

In [92]:
logreg_accuracy

0.8289940828402367

In [72]:
#82.7514% without x,y,z lat long handling
#with handling and replacement = 81.908%
# with handling and without replacement = 82.899%

#adding radians to the above without replacement of anything at all = 82.899%
#adding radians did nothing so we won't retain it